In [1]:
# Cell 1: Cài đặt đồng bộ
!pip install -q -U torch torchvision torchaudio transformers peft datasets bitsandbytes trl accelerate pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288

## Local Inference on GPU 
Model page: https://huggingface.co/Qwen/Qwen3-4B-Instruct-2507

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/Qwen/Qwen3-4B-Instruct-2507)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [2]:
import os
import gc
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from datasets import Dataset, DatasetDict

# Import Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    BitsAndBytesConfig, 
    TrainingArguments
)

# Import PEFT & TRL
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from sklearn.metrics import accuracy_score, classification_report

try:
    del model
    del trainer
    del tokenizer
except:
    pass
gc.collect()
torch.cuda.empty_cache()

# Environment config
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

2025-12-17 15:10:19.051923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765984219.232323      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765984219.282029      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [ ]:
# Load model
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
NEW_MODEL_NAME = "Llama-3.2-3B-ViANLI-Hallucination-Detection"

# Config data path
DATA_PATH = "/kaggle/input/processed/processed_data_3labels/ViANLI"
TRAIN_FILE = os.path.join(DATA_PATH, "train.csv")
DEV_FILE = os.path.join(DATA_PATH, "dev.csv")
TEST_FILE = os.path.join(DATA_PATH, "test.csv")

# Thiết lập device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Kiểm tra xem import có thành công không
print("Transformers version:", torch.__version__)

Using device: cuda
Transformers version: 2.9.1+cu128


In [ ]:
def load_and_process_data(train_path, dev_path, test_path):
    # Read CSV
    df_train = pd.read_csv(train_path)
    df_dev = pd.read_csv(dev_path)
    df_test = pd.read_csv(test_path)

    # 1. Định nghĩa Map từ Số sang Chữ
    label_map = {
        0: "Entailment",
        1: "Extrinsic-Hal",
        2: "Intrinsic-Hal"
    }

    # 2. Prompt chuyên gia NLI tiếng Việt
    def format_instruction(row):
        prompt = f"""You are an AI expert in Vietnamese Natural Language Inference (NLI). Your task is to determine the logical relationship between the Context and the Statement.

        Context: {row['sentence1']}
        Statement: {row['sentence2']}

        Based on the context, classify the statement as one of the following:
        - Entailment: The statement is definitely true based on the context.
        - Extrinsic-hal: The truth of the statement cannot be determined from the context (it might be true or false, but the context doesn’t say).
        - Intrinsic-hal: The statement is definitely false or contradicts the context.

        Answer:
        """
        return prompt

    def process_df(df):
        data = []
        for _, row in df.iterrows():
            # Kiểm tra dữ liệu lỗi
            if pd.isna(row['sentence1']) or pd.isna(row['sentence2']) or pd.isna(row['label']):
                continue
            
            # Lấy prompt
            user_content = format_instruction(row)
            
            # Xử lý nhãn: Chuyển từ số (0,1,2) sang chữ (entailment,...)
            try:
                label_idx = int(row['label'])
                if label_idx not in label_map:
                    continue # Bỏ qua nếu nhãn không phải 0,1,2
                assistant_content = label_map[label_idx]
            except ValueError:
                continue # Bỏ qua nếu nhãn không phải số

            messages = [
                {"role": "user", "content": user_content},
                {"role": "assistant", "content": assistant_content}
            ]

            data.append({"messages": messages, "label_str": assistant_content})
        return Dataset.from_list(data)

    print("Processing Train set...")
    train_dataset = process_df(df_train)
    print("Processing Dev set...")
    dev_dataset = process_df(df_dev)
    print("Processing Test set...")
    test_dataset = process_df(df_test)

    return DatasetDict({"train": train_dataset, "dev": dev_dataset, "test": test_dataset})

# Load data
dataset = load_and_process_data(TRAIN_FILE, DEV_FILE, TEST_FILE)
print("\nSample train data:", dataset['train'][0])

Processing Train set...
Processing Dev set...
Processing Test set...

Sample train data: {'messages': [{'content': 'You are an AI expert in Vietnamese Natural Language Inference (NLI). Your task is to determine the logical relationship between the Context and the Statement.\n\n        Context: Sau dự án điện mặt trời, Tập đoàn Trung Nam vừa bán 35,1% cổ phần Nhà máy Điện gió Trung Nam cho Công ty Hitachi Sustainable Energy.\n        Statement: Công ty Hitachi Sustainable Energy có cổ phần trong Nhà máy Điện gió Trung Nam.\n\n        Based on the context, classify the statement as one of the following:\n        - entailment: The statement is definitely true based on the context.\n        - neutral: The truth of the statement cannot be determined from the context.\n        - contradiction: The statement is definitely false or contradicts the context.\n\n        Answer:\n        ', 'role': 'user'}, {'content': 'entailment', 'role': 'assistant'}], 'label_str': 'entailment'}


In [5]:
# Config Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, 
    bnb_4bit_use_double_quant=False,
)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load Model (Ép kiểu float16 ngay từ đầu)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16 
)

model.config.use_cache = False 
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

# LoRA Config
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=16, 
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [6]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,   
    bf16=False, 
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none",
    gradient_checkpointing=True 
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['dev'],
    peft_config=peft_config,
    args=training_args,
    processing_class=tokenizer,
)

# --- BƯỚC VÁ LỖI (HYBRID CAST) ---
print("Applying Hybrid Cast for T4 compatibility...")
for name, param in trainer.model.named_parameters():
    if param.requires_grad:
        if param.dtype != torch.float32:
            param.data = param.data.to(torch.float32)
    elif param.dtype == torch.bfloat16:
        param.data = param.data.to(torch.float16)

for name, buffer in trainer.model.named_buffers():
    if buffer.dtype == torch.bfloat16:
        buffer.data = buffer.data.to(torch.float16)
print("Hybrid Cast complete.")

# Train
trainer.train()

# Save Model Local
trainer.model.save_pretrained(NEW_MODEL_NAME)
tokenizer.save_pretrained(NEW_MODEL_NAME)
print("Training finished and model saved.")

Tokenizing train dataset:   0%|          | 0/18282 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/18282 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2255 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/2255 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Applying Hybrid Cast for T4 compatibility...
Hybrid Cast complete.


Step,Training Loss
25,1.790500
50,0.821500
75,0.896600
100,0.704400
125,0.865800
150,0.684100
175,0.821800
200,0.645100
225,0.796600
250,0.627800


Training finished and model saved.


In [7]:
# --- 1. Dọn dẹp bộ nhớ ---
try:
    del model
    del trainer
except:
    pass
gc.collect()
torch.cuda.empty_cache()

# --- 2. Cấu hình ---
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"
NEW_MODEL_NAME = "Qwen-ViNLI-Hallucination-Detection"
ADAPTER_PATH = f"./{NEW_MODEL_NAME}"

# --- 3. Load Base Model ---
print("Loading Base Model...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

# --- 4. Load Adapter ---
print(f"Loading Adapter from {ADAPTER_PATH}...")
if os.path.exists(ADAPTER_PATH):
    model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
    model.eval()
else:
    raise FileNotFoundError(f"Không tìm thấy adapter tại {ADAPTER_PATH}. Hãy kiểm tra lại quá trình train.")

# --- 5. Load Tokenizer & Cấu hình Batch ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# QUAN TRỌNG: Khi generate theo batch, phải padding bên TRÁI
tokenizer.padding_side = "left" 

# --- 6. Hàm Batch Predict ---
def batch_predict(batch_messages, model, tokenizer):
    # 1. Apply template cho cả batch
    prompts = [
        tokenizer.apply_chat_template(msg, tokenize=False, add_generation_prompt=True) 
        for msg in batch_messages
    ]
    
    # 2. Tokenize batch (Padding & Truncation)
    inputs = tokenizer(
        prompts, 
        return_tensors="pt", 
        padding=True, 
        truncation=True,
        max_length=2048 # Giới hạn độ dài để tránh OOM
    ).to("cuda")
    
    # 3. Generate
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs, 
            max_new_tokens=10, 
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 4. Lấy phần output (cắt bỏ phần input prompt)
    # input_ids.shape[1] là độ dài của prompt sau khi padding
    outputs = generated_ids[:, inputs.input_ids.shape[1]:]
    
    # 5. Decode
    responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return [r.strip() for r in responses]

# --- 7. Chạy Evaluation với Batch ---
y_true = []
y_pred = []

# Cấu hình Batch Size (T4 chạy ổn với 8 hoặc 16, nếu OOM thì giảm xuống 4)
BATCH_SIZE = 16 

print(f"Starting evaluation on Test set (Batch Size: {BATCH_SIZE})...")
test_subset = dataset['test'] 
total_samples = len(test_subset)

# Vòng lặp Batch
for i in tqdm(range(0, total_samples, BATCH_SIZE)):
    # Lấy một batch dữ liệu
    batch_data = test_subset[i : i + BATCH_SIZE]
    
    batch_messages = batch_data['messages']
    batch_ground_truth = batch_data['label_str']
    
    try:
        # Dự đoán cả lô
        batch_predictions = batch_predict(batch_messages, model, tokenizer)
        
        # Lưu kết quả
        y_true.extend(batch_ground_truth)
        y_pred.extend(batch_predictions)
        
    except RuntimeError as e:
        if "out of memory" in str(e):
            print(f"⚠️ OOM tại index {i}. Đang dọn dẹp cache và thử lại với batch nhỏ hơn...")
            torch.cuda.empty_cache()
            # Fallback: Chạy từng cái một cho batch bị lỗi này
            for j in range(len(batch_messages)):
                try:
                    single_pred = batch_predict([batch_messages[j]], model, tokenizer)[0]
                    y_true.append(batch_ground_truth[j])
                    y_pred.append(single_pred)
                except:
                    y_true.append(batch_ground_truth[j])
                    y_pred.append("error")
        else:
            print(f"Error at batch {i}: {e}")
            # Điền giá trị rỗng để không lệch index
            y_true.extend(batch_ground_truth)
            y_pred.extend(["error"] * len(batch_ground_truth))

# --- 8. Đánh giá và Báo cáo (%) ---
def clean_pred(pred):
    pred = str(pred).lower().strip()
    if "entailment" in pred: return "entailment"
    if "contradiction" in pred: return "contradiction"
    if "neutral" in pred: return "neutral"
    return "unknown"

y_pred_clean = [clean_pred(p) for p in y_pred]

# Định nghĩa target names khớp với ViNLI (0,1,2 -> text)
target_names = ["entailment", "neutral", "contradiction"]

# Tạo báo cáo
report_dict = classification_report(y_true, y_pred_clean, labels=target_names, output_dict=True)
df_report = pd.DataFrame(report_dict).transpose()

# Format phần trăm
cols_to_scale = ['precision', 'recall', 'f1-score']
df_report[cols_to_scale] = df_report[cols_to_scale] * 100
df_report['support'] = df_report['support'].astype(int)

print("\nClassification Report (Unit: %):")
print("-" * 60)
print(df_report.to_string(float_format="{:.3f}".format))
print("-" * 60)

# Lưu kết quả
df_result = pd.DataFrame({
    "Ground_Truth": y_true, 
    "Prediction_Raw": y_pred, 
    "Prediction_Clean": y_pred_clean
})
df_result.to_csv("submission_results_vinli.csv", index=False)
print("Results saved to submission_results_vinli.csv")

Loading Base Model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading Adapter from ./Qwen-ViNLI-Hallucination-Detection...


FileNotFoundError: Không tìm thấy adapter tại ./Qwen-ViNLI-Hallucination-Detection. Hãy kiểm tra lại quá trình train.